In [1]:
import logging
import logging.config
import os
import json
# import ConfigParser
import sys

from simpleMenu import menu
import configuration
from gdrive.auth import getCredentials
from gdrive.gdrive import googledrive, GDriveError


# get the current working directory
# When launching a .command from the OS X Finder, the working directory is typically ~/; this is problematic
# for locating resource files
# I don't love this hack, but it works.
try:
    __file__
    cwd = os.path.dirname(__file__)+'/'
except NameError as e:
    cwd = os.getcwd()

In [2]:
def setup_logging(
    default_path='logging.json',
    default_level=logging.INFO,
    env_key='LOG_CFG'
):
    """Setup logging configuration

    """
    path = default_path
    value = os.getenv(env_key, None)
    if value:
        path = value
    if os.path.exists(path):
        with open(path, 'rt') as f:
            config = json.load(f)
        logging.config.dictConfig(config)
        logging.getLogger().setLevel(default_level)
    else:
        logging.basicConfig(level=default_level)

In [ ]:
help(configuration)

In [3]:
def getConfiguration(cfgfile):
    # required configuraiton options
    # Section: {'option': 'default value'}
    config_required = {
        'Main': {'credentials': os.path.join(cfgpath, 'credentials/'), 
                 'teamdriveID': '',
                 'folderID': '',
                 'teamdrivename': ''
                 },
        }

    config = configuration.get_config(cfgfile)

    update_config = False

    for section, values in config_required.items():
        if not config.has_section(section):
            logger.warn('section: {} not found in {}'.format(section, cfgfile))
            logger.debug('adding section {}'.format(section))
            config.add_section(section)
            update_config = True
        for option, value in values.items():
            if not config.has_option(section, option):
                logger.warn('option: {} not found in {}'.format(option, cfgfile))
                logger.debug('adding option {}: {}'.format(option, value))

                config.set(section, option, value)
                update_config = True


    # for section, options in config_required.items():

    if update_config:
        try:
            logger.info('updating configuration file at: {}'.format(cfgfile))
            configuration.create_config(cfgfile, config)
        except Exception as e:
            logger.error(e)
            
    return(config)

In [4]:
def getTeamDrive(myDrive):
    '''
    menu interaction to find appropriate teamdrive ID for storing portfolios
    '''
    logger = logging.getLogger(__name__)
    myMenu = menu()
    teamDrives = {}
    logging.debug('fetching list of team drives')
    for each in myDrive.listTeamDrives():
        try:
            teamDrives[each['name']] = each['id']
            myMenu.addItem(each['name'])
        except KeyError as e:
            logger.warn('KeyError: {}'.format(e))
            
    myMenu.sortMenu()
    td_name = myMenu.loopChoice(optional=True, message='Choose Team Drive for Portfolios')
    if not td_name == 'Q':
#         return(teamDrives[td_name])
        return([td_name, teamDrives[td_name]])
    else:
        return(None)
    

In [ ]:
foo = getTeamDrive(myDrive)
print len(foo)

In [ ]:
foo

In [7]:
def getPortfolioFolder(myDrive, teamdriveID):
    '''
    menu interaction to find appropriate folder for storing portfolios
    '''
    logger = logging.getLogger(__name__)
    myMenu = menu()
    print 'Searching in Team Drive: {}'.format(teamdriveID[0])
    
    
    logger.debug('')

In [8]:
getPortfolioFolder(myDrive, teamdriveID)

Searching in Team Drive: IT Blabla
2018-08-14 16:07:54,901: [DEBUG: __main__.getPortfolioFolder] 


In [12]:

version = '00.00 - 18.08.13'
appName = 'portfolio_creator'

cfgfile = appName+'.ini'
cfgpath = os.path.join('~/.config/', appName)
cfgfile = os.path.expanduser(os.path.join(cfgpath, cfgfile))
updateConfig = False

logger = logging.getLogger(__name__)
setup_logging(default_level=logging.DEBUG)
logging.info('===Starting {} Log==='.format(appName))

myConfig = getConfiguration(cfgfile)

####### set all config options 
teamdriveID[0] = myConfig.get('Main', 'teamdrivename')

    
#     , myConfig.get('Main', 'teamdriveid')]
# folderID = [myConfig.get('Main', folderid), myConfig.get('Main', 'foldername')]


if logging.getLogger().getEffectiveLevel() <= 10:
    config_dict = {}
    for section in myConfig.sections():
        config_dict[section] = {}
        for option in myConfig.options(section):
            config_dict[section][option] = myConfig.get(section, option)

    logging.debug('current configuration:')
    logging.debug('\n{}'.format(config_dict))

# print '\n'*10
print 'Welcome to the portfolio creator for Google Team Drive version {0}'.format(version)
print 'This program will create portfolio folders in Google Team Drive for students.'
print 'You will need a student_export.text file from PowerSchool with at least the following information:'
print '     ClassOf, FirstLast, Student_Number\n'
print 'The order of the CSV does not matter, but the headers must be on the very first line.'
# raw_input("Press Enter to continue...\n\n\n")

    
# google drive authorization
logging.info('checking google credentials')
#### Add instructions before this shoots user over to a web authorization 
credential_store = os.path.expanduser(myConfig.get('Main', 'credentials'))
try:
    credentials = getCredentials(credential_store)
except Exception as e:
    logging.critical(e)

# configure google drive object
myDrive = googledrive(credentials)
    
if len(teamdriveID) < 1:
    # set up team drive id
    teamdriveID = getTeamDrive(myDrive)
    if not teamdriveID:
        logging.debug('user exit')
        sys.exit(0)
    else:
        myConfig.set('Main', 'teamdrivename', teamdriveID[0])
        myConfig.set('Main', 'teamdriveid', teamdriveID[1])
        updateConfig = True
        


    if updateConfig:
        configuration.create_config(cfgfile, myConfig)




2018-08-14 16:14:32,021: [INFO: root.<module>] ===Starting portfolio_creator Log===
2018-08-14 16:14:32,022: [DEBUG: configuration.get_config] reading configuration file at: /Users/aaronciuffo/.config/portfolio_creator/portfolio_creator.ini


NameError: name 'folderid' is not defined

In [ ]:
foo = myDrive.search(teamdrive = teamdriveID, fuzzy = True, name = 'Port')
# help(myDrive)
print foo

In [11]:
teamdriveID


['IT Blabla', '0ACLfU8KeD_BHUk9PVA']